In [7]:
import phoenix as px

session = px.launch_app()

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [13]:
# if you want to view in ipynb
px.active_session().view()

📺 Opening a view to the Phoenix app. The app is running at http://localhost:6006/


In [9]:
from openinference.instrumentation.dspy import DSPyInstrumentor
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

endpoint = "http://127.0.0.1:6006/v1/traces"
resource = Resource(attributes={})
tracer_provider = trace_sdk.TracerProvider(resource=resource)
span_otlp_exporter = OTLPSpanExporter(endpoint=endpoint)
tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter=span_otlp_exporter))

trace_api.set_tracer_provider(tracer_provider=tracer_provider)
DSPyInstrumentor().instrument()

In [10]:
import dspy

lm = dspy.OpenAI()
dspy.configure(lm=lm)

simpleqa = dspy.ChainOfThought("question -> answer")

In [11]:
answer = simpleqa(question="When is last Crusades begin? and Who won?").answer
print(answer)

The last Crusades began in 1270 and ended in 1291. The Mamluks (Muslims) won the last Crusades.


In [12]:
print(session.url)

http://localhost:6006/
